Считываем данные из 'ruwiki-latest-abstract.xml' и разбиваем на строки

In [ ]:
with open('ruwiki-latest-abstract.xml', encoding= 'utf-8') as file:
    file_row = [row.strip() for row in file]
file_row

Проходим по строкам и выбираем только ссылки
Сохраняем в массив urls

In [ ]:
import string
urls = []
for row in file_row:
    if "<url>" in row:
        urls.append(row[5:-6])
        
urls   

Переходим на страницу википедии по каждой ссылке, проверяем есть ли идентификатор "DrugBank" в инфобоксе (или в одном из инфобоксов, если их несколько)
Подходящие статьи записываем в файл 'DrugBank_article.txt'

In [ ]:
import multiprocess
import requests

def my_f(url):
    import requests
    import string
    from bs4 import BeautifulSoup
    req = requests.get(url) 
    soup = BeautifulSoup(req.text)
    all_infobox = soup.findAll("table", {"class": "infobox"})
    for i in all_infobox:
        if "DrugBank" in i.text:
            if i.find("a", href= re.compile('h*//www.drugbank.ca/*')).string != "none" and i.find("a", href= re.compile('h*//www.drugbank.ca/*')) != "?":
                with open('DrugBank_article.txt', 'a', encoding = "utf-8") as filehandle:  
                    filehandle.write('%s\n' % req.text)
        break
        
for i in urls:
        p = multiprocess.Process(target=my_f, args=(i, ))
        p.start()

print("end")


Считываем статьи из файла 'DrugBank_article.txt', записываем в массив drug_article

In [26]:
filehandle = open('DrugBank_article.txt', 'r', encoding="utf-8").read() 
drug_article = filehandle.split("<!DOCTYPE html>")
drug_article=drug_article[1::]


Длина массива drug_article (=942)

In [27]:
len(drug_article)

942

In [ ]:
Импорт библиотек

In [1]:
from bs4 import BeautifulSoup
import re
import string

Проходим по статьям из drug_article и формируем строку таблицы str 
Строку записываем в файл 'drugBank_table.txt'

In [ ]:
for article in drug_article:
    soup = BeautifulSoup(article)
    all_infobox = soup.findAll("table", {"class": "infobox"})
    for i in all_infobox:
        if "DrugBank" in i.text:
            infobox = i
    name = soup.find("h1", id="firstHeading").string
    if name is None:
        name = infobox.find("th", {"class":"infobox-above"}).string
    str = ""
    str = str + soup.find("link", {"rel": "canonical"})["href"] + "\\t" + name + "\\t" + infobox.find("a", href= re.compile('h*//www.drugbank.ca/*')).string + "\\t" + infobox.find("a", href= re.compile('h*://www.drugbank.ca/*'))["href"] 
    if "Другие названия" in infobox.text:
        str = str + "\\t" + infobox.findAll("td", {"class": "plainlist"})[-1].text[1::].replace("\n", " ")
    with open('drugBank_table.txt', 'a', encoding="utf-8") as filehandle:  
        filehandle.write('%s\n' % str)